In [138]:
import json
import os
from os import listdir
import matplotlib.pyplot as plt
import datetime, time
import pytz
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


In [139]:
csv_files = os.listdir("D:/Git/Data_for_project5/Part1E_data_train/")
print(csv_files)
features = ["num_of_tweets","num_of_retweets","Sum of followers"," max of followers","hour","count of favorites","sum of verified","sum of status","sum of friends","sum of ranking scores","sum of impressions","sum of momentum"]

for csv_file in csv_files:
    #print("Doing OLS for " + csv_file[10:-4])
    #print("**************************************************************")
    df = pd.read_csv("D:/Git/Data_for_project5/Part1E_data_train/" + csv_file, index_col = 0)
    print(df.head())
    
    print("Reading CSV 1 done")
    final_df = df.copy()
    print(final_df.shape)
    break
for i in range(1,len(csv_files)):
    df = pd.read_csv("D:/Git/Data_for_project5/Part1E_data_train/" + csv_files[i],index_col = 0)
    final_df = pd.concat([final_df,df],axis = 0)
    print(final_df.shape)
    
df["time_of_tweet"] = [i[:-6] for i in df["time_of_tweet"]]
df['time_of_tweet'] = pd.to_datetime(df['time_of_tweet'])

['df_train_part1e_tweets_#gohawks.csv', 'df_train_part1e_tweets_#gopatriots.csv', 'df_train_part1e_tweets_#nfl.csv', 'df_train_part1e_tweets_#patriots.csv', 'df_train_part1e_tweets_#sb49.csv', 'df_train_part1e_tweets_#superbowl.csv']
               time_of_tweet  num_of_tweets  num_retweets  sum_followers  \
0  2014-12-28 14:14:35-08:00            1.0           5.0         1752.0   
1  2014-12-29 06:40:08-08:00            1.0           2.0          258.0   
2  2014-12-29 07:27:13-08:00            1.0           7.0          199.0   
3  2014-12-29 11:27:20-08:00            1.0           5.0           22.0   
4  2014-12-29 12:10:38-08:00            1.0           2.0           22.0   

   max_followers  sum_fav_count  sum_verified  sum_status  sum_friends  \
0         1752.0            1.0           0.0     12477.0       2025.0   
1          258.0            0.0           0.0       156.0         29.0   
2          199.0            0.0           0.0       552.0        190.0   
3           2

In [140]:
def make_train(df):
    hourly_df = df.groupby(pd.TimeGrouper(key = 'time_of_tweet',freq = '60Min'))
    X_train = np.zeros((len(hourly_df),12))
    Y_train = np.zeros((len(hourly_df)))
    for i,(hour,grp) in enumerate(hourly_df):
        X_train[i,0] = grp.num_of_tweets.sum()
        X_train[i,1] = grp.num_retweets.sum()
        X_train[i,2] = grp.sum_followers.sum()
        X_train[i,3] = grp.max_followers.max()
        X_train[i,4] = hour.hour
        X_train[i,5] = grp.sum_fav_count.sum()
        X_train[i,6] = grp.sum_verified.sum()
        X_train[i,7] = grp.sum_status.sum()
        X_train[i,8] = grp.sum_friends.sum()
        X_train[i,9] = grp.sum_ranking_score.sum()
        X_train[i,10] = grp.sum_impressions.sum()
        X_train[i,11] = grp.sum_momentum.sum()

        Y_train[i] = grp.num_of_tweets.sum()
    Y_train = Y_train[1:]
    X_train = X_train[:-1]
    X_train = np.nan_to_num(X_train)
    return X_train,Y_train

In [141]:
def make_test(df):
    hourly_df = df.groupby(pd.TimeGrouper(key = 'time_of_tweet',freq = '60Min'))
    X_train = np.zeros((len(hourly_df),12))
    Y_train = np.zeros((len(hourly_df)))
    for i,(hour,grp) in enumerate(hourly_df):
        X_train[i,0] = grp.num_of_tweets.sum()
        X_train[i,1] = grp.num_retweets.sum()
        X_train[i,2] = grp.sum_followers.sum()
        X_train[i,3] = grp.max_followers.max()
        X_train[i,4] = hour.hour
        X_train[i,5] = grp.sum_fav_count.sum()
        X_train[i,6] = grp.sum_verified.sum()
        X_train[i,7] = grp.sum_status.sum()
        X_train[i,8] = grp.sum_friends.sum()
        X_train[i,9] = grp.sum_ranking_score.sum()
        X_train[i,10] = grp.sum_impressions.sum()
        X_train[i,11] = grp.sum_momentum.sum()

        Y_train[i] = grp.num_of_tweets.sum()
    X_train = np.nan_to_num(X_train)
    return X_train,Y_train

In [142]:
X,Y = make_train(df)
print(len(X))

df1 = df[df.time_of_tweet < datetime.datetime(2015,2,1,8,0,0)]
print(df1.shape)
df2 = df[(df.time_of_tweet >= datetime.datetime(2015,2,1,8,0,0)) & (df.time_of_tweet <= datetime.datetime(2015,2,1,20,0,0))]
print(df2.shape)
df3 = df[df.time_of_tweet > datetime.datetime(2015,2,1,20,0,0)]
print(df3.shape)

X1,Y1 = make_train(df1)
X2,Y2 = make_train(df2)
X3,Y3 = make_train(df3)

963
(215075, 12)
(1027317, 12)
(106375, 12)


In [143]:
print(len(X1))
print(len(Y1))

print(len(X2))
print(len(Y2))

print(len(X3))
print(len(Y3))

print(len(X))
print(len(Y))

X1_new = []
Y1_new = []

for i in range(0,len(X1)):
    if i + 5 >= len(X1):
        break
    temp1 = X1[i:i+5,].copy()
    temp1 = temp1.ravel()
    #print(temp1.shape)
    X1_new.append(temp1)
    Y1_new.append(X1[i+5][0])
X2_new = []
Y2_new = []

for i in range(0,len(X2)):
    if i + 5 >= len(X2):
        break
    temp2 = X2[i:i+5,].copy()
    temp2 = temp2.ravel()
    
    X2_new.append(temp2)
    Y2_new.append(X2[i+5][0])
X3_new = []
Y3_new = []

for i in range(0,len(X3)):
    if i + 5 >= len(X3):
        break
    temp3 = X3[i:i+5,].copy()
    temp3 = temp3.ravel()
    X3_new.append(temp3)
    Y3_new.append(X3[i+5][0])

X_new = []
Y_new = []

for i in range(0,len(X)):
    if i + 5 >= len(X):
        break
    temp = X[i:i+5,].copy()
    temp = temp.ravel()
    X_new.append(temp)
    Y_new.append(X[i+5][0])

    

816
816
11
11
134
134
963
963


In [144]:
model_full = RandomForestRegressor(n_estimators = 20, max_depth = 5)
model1 = RandomForestRegressor(n_estimators = 20,max_depth = 5)
model2 = RandomForestRegressor(n_estimators = 20,max_depth = 5)
model3 = RandomForestRegressor(n_estimators = 20,max_depth = 5)
#print(X1_new[0:5])
X1_train = X1_new.copy()
X2_train = X2_new.copy()
X3_train = X3_new.copy()

Y1_train = Y1_new.copy()
Y2_train = Y2_new.copy()
Y3_train = Y3_new.copy()

X_train = X_new.copy()
#print(X_train[0:5])
Y_train = Y_new.copy()
print(len(X1_train))
print(len(X1_train[0]))
print(len(Y1_train))
print(len(X2_train))
print(len(X2_train[0]))
print(len(Y2_train))
print(len(X3_train))
print(len(X3_train[0]))
print(len(Y3_train))
print(len(X_train))
print(len(X_train[0]))
print(len(Y_train))
model1.fit(np.asarray(X1_train),np.asarray(Y1_train))
model2.fit(np.asarray(X2_train), np.asarray(Y2_train))
model3.fit(np.asarray(X3_train), np.asarray(Y3_train))
model_full.fit(np.asarray(X_train), np.asarray(Y_train))

811
60
811
6
60
6
129
60
129
958
60
958


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [145]:
csv_files = os.listdir("D:/Git/Data_for_project5/Part1E_data_test/")
print(csv_files)
features = ["num_of_tweets","num_of_retweets","Sum of followers"," max of followers","hour","count of favorites","sum of verified","sum of status","sum of friends","sum of ranking scores","sum of impressions","sum of momentum"]

for csv_file in csv_files:
    #print("Doing OLS for " + csv_file[10:-4])
    #print("**************************************************************")
    df = pd.read_csv("D:/Git/Data_for_project5/Part1E_data_test/" + csv_file, index_col = 0)
    df["time_of_tweet"] = [i[:-6] for i in df["time_of_tweet"]]
    df['time_of_tweet'] = pd.to_datetime(df['time_of_tweet'])
    
    print(csv_file[10:18])
    interval = int(csv_file[-5])
    print("Interval = " + str(interval))
    if interval == 1:
        X_test_1, Y_test_1 = make_test(df)
        #print(X_test_1[-5:])
        print(X_test_1.shape)
        X_test_new_1 = []
        #Y_new = []

        for i in range(0,len(X_test_1)):
            if i + 4 >= len(X_test_1):
                break
            temp1 = X_test_1[i:i+5,].copy()
            temp1 = temp1.ravel()

            X_test_new_1.append(temp1)
            #Y_new.append(X[i+5][0])
        X_test_new_1 = np.asarray(X_test_new_1)
        print(X_test_new_1.shape)
        preds = model1.predict(X_test_new_1)
        print(preds)
        preds = model_full.predict(X_test_new_1)
        print(preds)
        print(Y_test_1)
        #plt.scatter(preds,Y_new)
    elif interval == 2:
        X_test_2,Y_test_2 = make_test(df)
        print(X_test_2.shape)
        X_test_new_2 = []
        #Y_new = []

        for i in range(0,len(X_test_2)):
            if i + 4 >= len(X_test_2):
                break
            temp2 = X_test_2[i:i+5,].copy()
            temp2 = temp2.ravel()

            X_test_new_2.append(temp2)
            #Y_new.append(X[i+5][0])
        X_test_new_2 = np.asarray(X_test_new_2)
        print(X_test_new_2.shape)
        preds = model2.predict(X_test_new_2)
        print(preds)
        preds = model_full.predict(X_test_new_2)
        print(preds)
        print(Y_test_2)
        #plt.scatter(preds,Y_new)
    elif interval == 3:
        X_test_3, Y_test_3 = make_test(df)
        print(X_test_3.shape)
        X_test_new_3 = []
        #Y_new = []
        

        for i in range(0,len(X_test_3)):
            if i + 4 >= len(X_test_3):
                break
            temp3 = X_test_3[i:i+5,].copy()
            temp3 = temp3.ravel()

            X_test_new_3.append(temp3)
            #Y_new.append(X[i+5][0])
        X_test_new_3 = np.asarray(X_test_new_3)
        print(X_test_new_3.shape)
        preds = model3.predict(X_test_new_3)
        print(preds)
        preds = model_full.predict(X_test_new_3)
        print(preds)
        print(Y_test_3)
        #plt.scatter(preds,Y_new)
        
    #print(df.head())


['df_part1e_sample10_period3.csv', 'df_part1e_sample1_period1.csv', 'df_part1e_sample2_period2.csv', 'df_part1e_sample3_period3.csv', 'df_part1e_sample4_period1.csv', 'df_part1e_sample5_period1.csv', 'df_part1e_sample6_period2.csv', 'df_part1e_sample7_period3.csv', 'df_part1e_sample8_period1.csv', 'df_part1e_sample9_period2.csv']
sample10
Interval = 3
(6, 12)
(2, 60)
[19.91295455 20.59856061]
[19.58369129 23.96674623]
[62. 54. 68. 62. 58. 61.]
sample1_
Interval = 1
(6, 12)
(2, 60)
[182.66158937 204.1584097 ]
[140.27998125 186.74507106]
[137.  82.  68.  94. 171. 178.]
sample2_
Interval = 2
(6, 12)
(2, 60)
[111839.6  139914.85]
[125789.87857143 137591.42857143]
[ 7591.  9361. 10374. 20066. 81958. 82923.]
sample3_
Interval = 3
(6, 12)
(2, 60)
[698.67832431 558.6489206 ]
[510.11058076 448.42928797]
[441. 550. 610. 888. 616. 523.]
sample4_
Interval = 1
(6, 12)
(2, 60)
[318.65617763 251.92456719]
[323.78925337 283.08022961]
[419. 257. 236. 266. 267. 201.]
sample5_
Interval = 1
(6, 12)
(2, 60